#第13章：實務上的最佳實踐

#讓模型發揮最大效用

###13-1-1 超參數優化

In [1]:
!pip install keras-tuner -q

     |████████████████████████████████| 133 kB 5.4 MB/s 


####程式 13.1 KerasTuner模型建構函式

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

####程式 13.2 繼承HyperModel的子類別

In [3]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 1 Complete [00h 01m 06s]
val_accuracy: 0.971699982881546

Best val_accuracy So Far: 0.971699982881546
Total elapsed time: 00h 01m 06s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
32                |48                |units
rmsprop           |rmsprop           |optimizer

Epoch 1/100
391/391 - 2s - loss: 0.5022 - accuracy: 0.8676 - val_loss: 0.2659 - val_accuracy: 0.9244 - 2s/epoch - 4ms/step
Epoch 2/100
391/391 - 1s - loss: 0.2560 - accuracy: 0.9263 - val_loss: 0.2163 - val_accuracy: 0.9390 - 1s/epoch - 3ms/step
Epoch 3/100
391/391 - 1s - loss: 0.2138 - accuracy: 0.9380 - val_loss: 0.1940 - val_accuracy: 0.9453 - 1s/epoch - 3ms/step
Epoch 4/100
391/391 - 1s - loss: 0.1881 - accuracy: 0.9449 - val_loss: 0.1744 - val_accuracy: 0.9502 - 1s/epoch - 3ms/step
Epoch 5/100
391/391 - 1s - loss: 0.1689 - accuracy: 0.9513 - val_loss: 0.1622 - val_accuracy: 0.9536 - 1s/epoch - 3ms/step
Epoch 6/100
391/391 - 1s - loss: 0.1539 - accuracy: 0.9549 - val_loss: 0

KeyboardInterrupt: ignored

####程式13.3 查詢最佳的超參數配置

In [7]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [8]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [10]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 2s 4ms/step - loss: 0.4467 - accuracy: 0.8817 - val_loss: 0.2426 - val_accuracy: 0.9316
Epoch 2/100
391/391 [==============================] - 1s 4ms/step - loss: 0.2266 - accuracy: 0.9344 - val_loss: 0.1869 - val_accuracy: 0.9491
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1787 - accuracy: 0.9491 - val_loss: 0.1670 - val_accuracy: 0.9527
Epoch 4/100
391/391 [==============================] - 1s 4ms/step - loss: 0.1512 - accuracy: 0.9567 - val_loss: 0.1482 - val_accuracy: 0.9595
Epoch 5/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1309 - accuracy: 0.9626 - val_loss: 0.1345 - val_accuracy: 0.9639
Epoch 6/100
391/391 [==============================] - 1s 4ms/step - loss: 0.1159 - accuracy: 0.9666 - val_loss: 0.1257 - val_accuracy: 0.9630
Epoch 7/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1033 - accuracy: 0.9699 - val_loss: 0.1206 - val_accuracy: 0.9641

NameError: ignored

In [11]:
best_models = tuner.get_best_models(top_n)

##13-2 擴大模型的訓練規模

###13-2-1 使用混合精度加速GPU上的訓練

In [12]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

tf.float64

In [13]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

tf.float32

In [14]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


###13-2-3 在TPU上進行訓練

In [15]:
import tensorflow as tf 
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
print("Device:", tpu.master())

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.55.64.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.55.64.82:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


Device: grpc://10.55.64.82:8470


####程式 13.4 在TPUStrategy區塊中建立模型

In [18]:
keras.mixed_precision.set_global_policy("float32")

from tensorflow import keras
from tensorflow.keras import layers

strategy = tf.distribute.TPUStrategy(tpu)
print(f"Number of replicas:{strategy.num_replicas_in_sync}")

def build_model(input_size):
    inputs = keras.Input((input_size, input_size, 3))
    x = keras.applications.resnet.preprocess_input(inputs)
    x = keras.applications.resnet.ResNet50(
        weights=None, include_top=False, pooling="max")(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='rmsprop',
           loss="sparse_categorical_crossentropy",
           metrics=["accuracy"])
    return model

with strategy.scope():
    model = build_model(input_size=32)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of replicas:8


In [19]:
(x_train,y_train), (x_test,y_test) = keras.datasets.cifar10.load_data()
model.fit(x_train, y_train, batch_size=1024)

49/49 [==============================] - 73s 566ms/step - loss: 2.7688 - accuracy: 0.2598


## Summary